# 01 Attribute and Pickle the Network
Step 01: Add key attributes to the 'raw' network and create pickles

In [1]:
import os
import sys
import pickle

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O

In [5]:
input_dir = os.path.join("Z:/Data/Users/Sijia/Met_Council/client_met_council_network", "network_standard")
lasso_dir = os.path.join("C:/Users/wangs1/Lasso")
output_dir = input_dir

## Read Roadway and Transit Networks

In [6]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(link_file = link_file, node_file = node_file, shape_file = shape_file)

2021-07-20 16:33:06, INFO: Reading from following files:
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\link.json
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\node.geojson
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\shape.geojson.
2021-07-20 16:33:06, INFO: Reading from following files:
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\link.json
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\node.geojson
-Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\shape.geojson.
2021-07-20 16:38:00, INFO: Read 1134074 links from Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\link.json
2021-07-20 16:38:00, INFO: Read 1134074 links from Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard\link.json
2021-07-20 16:38:00, INFO: Read 354346 nodes from Z:/Data/Users/Sijia/Met_

In [7]:
roadway_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

In [8]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-07-20 16:41:27, INFO: Read in transit feed from: Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard
2021-07-20 16:41:27, INFO: Read in transit feed from: Z:/Data/Users/Sijia/Met_Council/client_met_council_network\network_standard
2021-07-20 16:41:27, INFO: Removing calendar.txt from transit network config because file not found
2021-07-20 16:41:27, INFO: Removing calendar.txt from transit network config because file not found
2021-07-20 16:41:27, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-07-20 16:41:27, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-07-20 16:41:28, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-07-20 16:41:28, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-07-20 16:41:28, INFO: Removing fare_rules.txt from transit network config because file not found
2021-07-20 1

## Keep the Drive Links
During the network updates, this allows for faster I/O. Also remove the legacy number of lanes coding, which is now computed via the `calculate_number_of_lanes` method.

In [9]:
# roadway_net.links_df = roadway_net.links_df[roadway_net.links_df['drive_access'] == 1]
roadway_net.links_df = roadway_net.links_df.drop('lanes', axis = 1)

## Attribute the Network

In [10]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-07-20 16:41:30, INFO: Lasso base directory set as: C:/Users/wangs1/Lasso
2021-07-20 16:41:30, INFO: Lasso base directory set as: C:/Users/wangs1/Lasso


In [11]:
roadway_net.links_df.drive_access.value_counts()

False    779982
True     354092
Name: drive_access, dtype: int64

In [12]:
r_net = metcouncil.calculate_number_of_lanes(
        roadway_net=roadway_net,
        parameters=parameters,
)
r_net.links_df.lanes.value_counts()

2021-07-20 16:41:30, INFO: Calculating Number of Lanes as network variable: 'lanes'
2021-07-20 16:41:30, INFO: Calculating Number of Lanes as network variable: 'lanes'
2021-07-20 16:41:30, INFO: Lasso base directory set as: C:/Users/wangs1/Lasso
2021-07-20 16:41:30, INFO: Lasso base directory set as: C:/Users/wangs1/Lasso
2021-07-20 16:41:30, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-07-20 16:41:30, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2021-07-20 16:41:30, INFO: Finished calculating centroid connector variable: centroidconnect
2021-07-20 16:41:30, INFO: Finished calculating centroid connector variable: centroidconnect
2021-07-20 16:44:32, INFO: Finished calculating number of lanes to: lanes
2021-07-20 16:44:32, INFO: Finished calculating number of lanes to: lanes


0    748890
1    356273
2     26560
3      1734
4       545
5        66
6         6
Name: lanes, dtype: int64

In [13]:
r_net = metcouncil.calculate_assign_group_and_roadway_class(
        roadway_net=r_net,
        parameters=parameters,
)
r_net.links_df.assign_group.value_counts()

2021-07-20 16:44:32, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2021-07-20 16:44:32, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2021-07-20 16:44:32, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2021-07-20 16:44:32, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2021-07-20 16:51:26, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class
2021-07-20 16:51:26, INFO: Finished calculating assignment group variable assign_group and roadway class variable roadway_class


101    748615
50     223143
9       55226
7       54336
6       44241
4        4384
2        1404
1        1388
15        937
98        185
11        123
100        90
8           2
Name: assign_group, dtype: int64

In [14]:
r_net.links_df.roadway_class.value_counts()

101    748615
50     224774
40      58710
99      55226
30      25958
20      14812
60       4384
10       1390
70        115
100        90
Name: roadway_class, dtype: int64

## Create a Scenario

In [15]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}

In [16]:
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2021-07-20 16:51:28, INFO: Creating Scenario
2021-07-20 16:51:28, INFO: Creating Scenario


## Make Pickles

In [17]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_00.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))